# Exercise 1 - Getting Started with Azure ML

In a separate browser tab, sign into your Azure subscription and view your portal at https://portal.azure.com. 

As you proceed through the tasks below, you'll toggle between this notebook and the portal to visually confirm that the code you've run here has had the intended results in your Azure subscription.

## Task 1: Install the Azure ML SDK for Python

The Azure ML SDK for Python provides classes you can use to work with Azure ML in your Azure subscription. Run the cell below to install the **azureml-sdk** Python package, including the optional *notebooks* component; which provides some functionality specific to the Jupyter Notebooks environment.

> **More Information**: For more details about installing the Azure ML SDK and its optional components, see the [Azure ML SDK Documentation](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)

In [2]:
#!pip install --upgrade azureml-sdk[notebooks,automl,explain]

import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.0.83


## Task 2: Sign Into your Azure Subscription

Now that you've installed the SDK, you can use it to create, manage, and use Azure ML related objects in your Azure subscription, which means you'll need an authenticated connection between the code in this notebook and your Azure subscription. To create this authenticated connection, you can use the **authentication** module in the Azure ML SDK. In this case, you'll use the **InteractiveLoginAuthentication** class to generate a session token.

Run the cell below, and when prompted, click the https://microsoft.com/devicelogin link and enter the automatically generated code. Then, sign into your Azure subscription in the browser tab that is opened. After you have successfully signed in, you can close the browser tab that was opened and return to this notebook.

In [3]:
from azureml.core import authentication

auth = authentication.InteractiveLoginAuthentication()
print('Signed in')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DZ4SUQZ7C to authenticate.


WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/fcb2b37b-5da0-466b-9b83-0014b67a7c78', 'tenant_id': 'fcb2b37b-5da0-466b-9b83-0014b67a7c78'}' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: ca471377-6ecb-4ddf-8419-4b0cc5a63700\r\nCorrelation ID: e160a90c-64ad-41f0-8487-d3b76eada49a\r\nTimestamp: 2020-02-06 23:47:52Z","error_codes":[50076],"timestamp":"2020-02-06 23:47:52Z","trace_id":"ca471377-6ecb-4ddf-8419-4b0cc5a63700","correlation_id":"e160a90c-64ad-41f0-8487-d3b76eada49a","error_uri":"https://login.microsoftonline.com/error?code=50076","suberror":"basic_action"}'


Interactive authentication successfully completed.
Signed in


## Task 3: Create or Check Workspace

A workspace is a centralized place to manage all of the Azure ML resources you need to work on a machine learning project.

It can be created via: 
- Visual Interface in the Azure Portal
- Azure CLI 
- Azure ML SDK 

In the code below, enter appropriate values for the 
- *SUBSCRIPTION_ID*, 
- *RESOURCE_GROUP*, 
- *WORKSPACE_NAME*, and 
- *REGION*

(You can find your Azure subscription ID in the Azure portal - just click the &#128273; **Subscriptions** tab on the left and then select the subscription you want to use). Then run the cell to create your workspace.

> **Note**: If you hadn't previously created an authenticated session, you'd automatically be prompted to sign into your Azure subscription!

In [ ]:
from azureml.core import Workspace

SUBSCRIPTION_ID = 'subscriptionid' # Get this from the Azure portal
RESOURCE_GROUP_NAME  = 'rg' # Get this from the Azure portal
WORKSPACE_NAME  = 'sw-name' # Name of your choice - if it doesn't exist, it will be created
REGION = 'northeurope'# Or a region of your choice

ws = None
try:
    # Find existing workspace
    ws = Workspace(workspace_name=WORKSPACE_NAME,
                   subscription_id=SUBSCRIPTION_ID,
                   resource_group= RESOURCE_GROUP_NAME)
    print (ws.name, "found.")
except Exception as ex:
    # If workspace not found, create it
    print(ex.message)
    print("Attempting to create new workspace...")
    ws = Workspace.create(name=WORKSPACE_NAME, 
                      subscription_id=SUBSCRIPTION_ID,
                      resource_group=RESOURCE_GROUP_NAME,
                      create_resource_group=True,
                      location=REGION 
                     )
    print(ws.name, "created.")
finally:
    # Save the workspace configuration for later
    if ws != None:
        ws.write_config()
        print(ws.name, "saved.")

In the [Azure portal](https://portal.azure.com) you find the resource group you specified containing: 
- *Machine Learning Workspace* 
- *Storage Account* (where the workspace will store data, code, and other saved items)
- *AppInsights* instance (used to monitor the workspace)
- *KeyVault* instance (used to manage secure information).

In the code above, note that you used the **write_config** method to save the workspace configuration. This saved a JSON configuration file in a hidden folder named **.azureml**, which you can verify with the following cell.

In [ ]:
# Print the config.json file
with open("./.azureml/config.json","r") as f:
    print(f.read())

This saved configuration file enables you to easily obtain a reference to the workspace by simply loading it, as demonstrated in the following cell. Note that this method will prompt you to reauthenticate against your Azure subscription if your session has expired.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

## Task 4: Run an Experiment

Let's see how Azure ML can help track metrics from a simple experiment that uses Python code to examine some data.

In this case, you'll use a simple dataset that contains details of patients that have been tested for diabetes. You'll run an experiment to explore the data, extracting statistics, visualizations, and data samples. With the addition of a few lines, the code uses an Azure ML *experiment* to log details of the run.

In [ ]:
from azureml.core import Experiment, Run
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-experiment")
print("Starting experiment:", experiment.name)

# Start logging data from the experiment
run = experiment.start_logging()

# load the diabetes dataset
data = pd.read_csv('data/diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log("observations", row_count)

# Create box plots for each feature variable by the "diabetic" label and log them
num_cols = data.columns[:-1]
for col in num_cols:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    data.boxplot(column = col, by = "Diabetic", ax = ax)
    ax.set_title(col + ' by Diabetic')
    ax.set_ylabel(col)
    run.log_image(name = col, plot = fig)
plt.show()

# Create a list of mean diabetes pedigree per age and log it
mean_by_age = data[["Age", "DiabetesPedigree"]].groupby(["Age"]).mean().reset_index()
ages = mean_by_age["Age"].tolist()
pedigrees = mean_by_age["DiabetesPedigree"].tolist()
for index in range(len(ages)):
       run.log_row("Mean Diabetes Pedigree by Age", Age = ages[index],Diabetes_Pedigree = pedigrees[index])

# Save a sample of the data and upload it to the experiment output
data.sample(100).to_csv("sample.csv", index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = './sample.csv')

# Complete tracking and get link to details
run.complete()


## Task 5: View Experiment Results

After the experiment has been finished, you can view the results. Start by running the following cell:

In [ ]:
run

*Don't worry if the status is still **Running**, it can take a while to update. Eventually it will be set to **Completed**.*

Note that the experiment has been assigned a unique ID, and the output includes a link to a details page in the Azure portal. Click this link to open a new browser tab and view the experiment run details, noting the following:

On the **Details** tab:

- The **Tracked Metrics** list includes the *observations* value (the number of records in the dataset), an image for each matplotlib plot that was generated, and a 
table for the mean diabetes pedigree by age.
- The *Mean Diabetes Pedigree by Age* table is plotted as a chart.
- Each matplotlib plot image is shown.

On the **Outputs** tab:

- The ouputs generated by the experiment are listed - including each of the plot images and a CSV file containing a sample of the data used in the experiment.